# Predicción posición Tierra y Luna futura

Comenzando con la posición de la Tierra y la Luna en 2000-01-01 00:00:00 referida al baricentro del sistema solar (SSB), determine la posición predicha para estos dos cuerpos usando la teoría del problema de los dos cuerpos para la fecha 2000-01-31 00:00:00 (30 días después).  Las posiciones finales deben calcularse también con relación al SSB.  Para ello debe usarse el valor provisto por Horizons para la posición del baricentro del sistema Tierra-Luna en la fecha final (es decir, en lugar de calcular analíticamente la posición del centro de masa del sistema se debe usar la provista por Horizons).

In [2]:
# Datos de la Luna y de la Tierra
mu_luna=4902.800066 # km^3/s^2
mu_tierra=398600.435436 # km^3/s^2
G=6.67408e-20 # km^3/(kg s^2)

mluna=mu_luna/G
mtierra=mu_tierra/G
mluna,mtierra

(7.346031312180854e+22, 5.972365261369357e+24)

## Posiciones y velocidades

In [3]:
#Luna
from numpy import array
r_luna=array([-2.659657952880802E+07,1.442683561835989E+08,6.678400538153946E+04])
v_luna=array([-2.926974096953943E+01,-6.020397947517719E+00,-1.740793458484102E-03])
#Tierra
r_tierra=array([-2.627892928682480E+07,1.445102393586391E+08,3.022818135935813E+04])
v_tierra=array([-2.983052803283506E+01,-5.220465685407924E+00,-1.014621798034465E-04])

## 1. Centro masa

In [4]:
#Centro de masa
r_cm=(mluna*r_luna+mtierra*r_tierra)/(mluna+mtierra)
v_cm=(mluna*v_luna+mtierra*v_tierra)/(mluna+mtierra)
M=mtierra+mluna
M,r_cm,v_cm

(6.045825574491166e+24,
 array([-2.62827889e+07,  1.44507300e+08,  3.06723560e+04]),
 array([-2.98237141e+01, -5.23018533e+00, -1.21381013e-04]))

## 2. Vectores relativos

In [5]:
rvec=r_tierra-r_luna
vvec=v_tierra-v_luna

rvec,vvec

(array([317650.24198322, 241883.17504019, -36555.82402218]),
 array([-0.56078706,  0.79993226,  0.00163933]))

In [6]:
# Unidades
UL = 3.83e5 # km
UM = mtierra+mluna
from numpy import sqrt
UT = sqrt(UL**3/(G*UM))
UV = UL/UT
mu=1
UT/86400,UV

(4.318778271891243, 1.0264176790972799)

In [7]:
rvec=rvec/UL
vvec=vvec/UV

rvec,vvec

(array([ 0.829374  ,  0.63154876, -0.09544602]),
 array([-0.54635367,  0.77934381,  0.00159714]))

## 3. Constantes

In [8]:
# Vectores clave
from numpy import cross
from numpy.linalg import norm
r=norm(rvec)
hvec = cross(rvec,vvec)
evec = cross(vvec,hvec)-rvec/r
nvec = cross([0,0,1],hvec)

## 4. Parámetros de la órbita

In [9]:
h=norm(hvec)
e=norm(evec)

p=h**2/mu
a=p/(1-e**2)

hvec,evec,nvec,e,p*UL,a*UL

(array([0.07539393, 0.05082266, 0.99141647]),
 array([-0.01970893, -0.06151967,  0.00465246]),
 array([-0.05082266,  0.07539393,  0.        ]),
 0.06476694137438452,
 379619.56864553835,
 381218.68829050136)

## 5. Elementos orbitales

In [10]:
# Orientación de la órbita
from numpy import arccos,dot,pi
ip=arccos(dot(hvec,[0,0,1])/h)
i=ip
Wp=arccos(dot(nvec,[1,0,0])/norm(nvec))
W=Wp if nvec[1]>0 else 2*pi-Wp
wp=arccos(dot(nvec,evec)/(norm(nvec)*norm(evec)))
w=wp if evec[2]>=0 else 2*pi-wp
fp=arccos(dot(rvec,evec)/(norm(rvec)*norm(evec)))
fo=fp if dot(rvec,vvec)>0 else 2*pi-fp

fo*180/pi

145.15503092423884

## 6. Anomalía media inicial

In [11]:
#Anomalía excéntrica
from numpy import tan,arctan,sqrt
Eo=2*arctan(sqrt((1-e)/(1+e))*tan(fo/2))

Eo*180/pi

142.9748179782127

In [12]:
# Anomalía media inicial: Mo = Eo - e sin Eo
from numpy import sin
Mo=Eo-e*sin(Eo)

Mo*180/pi

140.74025689441984

## 7. Anomalía media futura

In [13]:
# Tiempo: 30 días después
t=30*86400/UT

In [14]:
# Movimiento orbital medio del sistema
n=sqrt(mu/a**3)

n

1.007017196507007

In [15]:
#Anomalía media futura
M=Mo+n*(t-0)

M*180/pi

541.5330155720809

In [16]:
#Entre 0 y 360 grados
from numpy import mod
M=mod(M,2*pi)

M*180/pi

181.5330155720809

## 8. Solución a la ecuación de Kepler

In [17]:
#Anomalía excéntrica
from pymcel.export import kepler_newton
E,error,ni=kepler_newton(M,e,1e-8)

E*180/pi

181.43977552700463

In [19]:
#Anomalía verdadera
f=2*arctan(sqrt((1+e)/(1-e))*tan(E/2))

f,f*180/pi

(-3.1180417496790733, -178.65063260219762)

In [22]:
360-178.65063260219762

181.34936739780238

## 9. Cálculo de la posición y la velocidad a partir de los elementos orbitales

In [24]:
from pymcel.export import elementos_a_estado
x=elementos_a_estado(mu,array([p,e,i,W,w,f]))
r=x[:3]
v=x[3:]

r,v

(array([ 0.29871367,  1.01407799, -0.07470053]),
 array([-0.89893809,  0.26722291,  0.0546627 ]))

## 10. Estado en el sistema de referencia original

In [26]:
# Posiciones de la Tierra y la Luna respecto al CM
rt_tierra=mluna/(mtierra+mluna)*r
vt_tierra=mluna/(mtierra+mluna)*v

rt_luna=-mtierra/(mtierra+mluna)*r
vt_luna=-mtierra/(mtierra+mluna)*v

rt_tierra,vt_tierra,rt_luna,vt_luna

(array([ 0.00362955,  0.01232164, -0.00090766]),
 array([-0.01092262,  0.00324691,  0.00066418]),
 array([-0.29508413, -1.00175635,  0.07379287]),
 array([ 0.88801547, -0.263976  , -0.05399851]))

In [28]:
# Posición del centro de masa del sistema en t = 2000-01-31
rcm=array([-9.658268525168775E+07,1.117579562771948E+08,3.035647087994218E+04])
vcm=array([-2.315623122669038E+01,-1.943555800686110E+01,-1.239056982322850E-04])

rcm,vcm

(array([-9.65826853e+07,  1.11757956e+08,  3.03564709e+04]),
 array([-2.31562312e+01, -1.94355580e+01, -1.23905698e-04]))

In [32]:
# Calculamos ahora la posición respecto al SSB
rt_tierra_SSB=rcm+rt_tierra*UL
vt_tierra_SSB=vcm+vt_tierra*UV

rt_luna_SSB=rcm+rt_tierra*UL
vt_luna_SSB=vcm+vt_tierra*UV

rt_tierra_SSB,vt_tierra_SSB,rt_luna_SSB,vt_luna_SSB

(array([-9.65812951e+07,  1.11762675e+08,  3.00088390e+04]),
 array([-2.31674424e+01, -1.94322253e+01,  5.57824214e-04]),
 array([-9.65812951e+07,  1.11762675e+08,  3.00088390e+04]),
 array([-2.31674424e+01, -1.94322253e+01,  5.57824214e-04]))

Ahora comparamos con lo que nos entrega Horizons:

Tierra: 

X =-9.658098596353926E+07 Y = 1.117625598418280E+08 Z = 2.998860558517277E+04

VX=-2.316716073476717E+01 VY=-1.943129637946240E+01 VZ= 4.998724441378499E-04
 
Luna
 
X =-9.672083834518087E+07 Y = 1.113836838527478E+08 Z = 6.026412868751585E+04

VX=-2.226765600034653E+01 VY=-1.978203073955845E+01 VZ=-5.083742364905941E-02

In [34]:
rt_tierra_h=array([-9.658098596353926E+07,1.117625598418280E+08,2.998860558517277E+04])
vt_tierra_h=array([-2.316716073476717E+01,-1.943129637946240E+01,4.998724441378499E-04])

rt_luna_h=array([-9.672083834518087E+07,1.113836838527478E+08,6.026412868751585E+04])
vt_luna_h=array([-2.226765600034653E+01,-1.978203073955845E+01,-5.083742364905941E-02])

In [36]:
rcm_h=(mtierra*rt_tierra_h+mluna*rt_luna_h)/(mtierra+mluna)
vcm_h=(mtierra*vt_tierra_h+mluna*vt_luna_h)/(mtierra+mluna)

rcm_h,vcm_h

(array([-9.65826853e+07,  1.11757956e+08,  3.03564709e+04]),
 array([-2.31562312e+01, -1.94355580e+01, -1.23905698e-04]))